# **Su Kalitesi**

[Kaynak](https://www.kaggle.com/jaykumar1607/water-quality-analysis-plotly-and-modelling)

1. **ph**: Suyun pH'ı (0 ile 14).

2. **Hardness (Sertlik)**: Suyun sabunu çökeltme kapasitesi mg/L olarak. 

3. **Solids (Katılar)**: ppm cinsinden toplam çözünmüş katılar.

4. **Chloramines (Kloraminler)**: Kloraminlerin ppm cinsinden miktarı.

5. **Sulfate (Sülfat)**: mg/L cinsinden çözünen Sülfat miktarı. 

6. **Conductivity (İletkenlik)**: μS/cm cinsinden suyun elektriksel iletkenliği. 

7. **Organic_carbon (Organik Karbon)**: ppm cinsinden organik karbon miktarı. 

8. **Trihalomethanes (Trihalometanlar)**: μg/L cinsinden Trihalometan miktarı. 

9. **Turbidity (Bulanıklık)**: NTU'da suyun ışık yayma özelliğinin ölçüsü.

10. **Potability (İçilebilirlik)**: Suyun insan tüketimi için güvenli olup olmadığını gösterir. İçilebilir : 1 / İçilebilir değil : 0 

## **Kütüphaneler**

In [ ]:
# Temel

import numpy as np
import pandas as pd
from collections import Counter

from warnings import filterwarnings
filterwarnings("ignore")

# Görselleştirme

import seaborn as sns
import plotly.express as px
import missingno as msno

# Veri Ön İşleme

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

# Modelleme

from sklearn.svm import SVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier, Perceptron, RidgeClassifier, PassiveAggressiveClassifier

# Değerlendirme & CV

from sklearn.metrics import precision_score, accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold, RandomizedSearchCV

## **Renkler**

In [ ]:
colors_blue = ["#132C33","#264D58","#17869E","#51C4D3","#B4DBE9"]
colors_dark = ["#1F1F1F","#313131","#636363","#AEAEAE","#DADADA"]
colors_green = ["#01411C","#4B6F44","#4F7942","#74C365","#D0F0C0"]

sns.palplot(colors_blue)
sns.palplot(colors_green)
sns.palplot(colors_dark)

## **Veriyi İçe Aktarmak**

In [ ]:
df = pd.read_csv("../input/water-potability/water_potability.csv")
df.head()

In [ ]:
df.info()

## **Görselleştirmeler**

In [ ]:
d = pd.DataFrame(df["Potability"].value_counts())

fig = px.pie(d, hole=0.4, opacity=0.6,
             color_discrete_sequence=[colors_green[3],colors_blue[3]],
             values="Potability",
             names=["İçilemez","İçilebilir"],
             labels={"Label":"Potability", "Potability":"miktar"})

fig.add_annotation(x=1.2, y=0.9, showarrow=False, font_size=12, opacity=0.7, font_family="monospace",
                   text="Dengeli bir veri kümesi elde etmek için<br> verileri yeniden örnekleyebiliriz")

fig.add_annotation(x=0.5, y=0.5, showarrow=False, font_size=14, opacity=0.7,
                   font_family="monospace", text="İçilebilirlik")

fig.update_layout(font_family="monospace",
    title=dict(text="İçilebilir su numunesi kaç tanedir?",
               x=0.47, y=0.98, font = dict(color=colors_dark[2], size=20)),
    legend=dict(x=0.37, y=-0.05, orientation="h", traceorder="reversed"),
    hoverlabel=dict(bgcolor="white"))

fig.update_traces(textposition="outside", textinfo="percent+label")

fig.show()

Özelliklerin dağılımını kontrol edelim.

* **Su Sertliği (Hardenss of water):** Su sertliğinin basit tanımı, suda çözünmüş kalsiyum ve magnezyum miktarıdır. Sert su, çözünmüş minerallerde, büyük ölçüde kalsiyum ve magnezyumda yüksektir. Sert suyun etkilerini, kelimenin tam anlamıyla, ellerinizi en son yıkadığınızda hissetmiş olabilirsiniz. Suyunuzun sertliğine bağlı olarak, yıkamak için sabun kullandıktan sonra ellerinizde bir kalıntı tabakası kalmış gibi hissedebilirsiniz. Sert suda sabun, kalsiyumla (sert suda nispeten yüksek olan) reaksiyona girerek "sabun köpüğü" oluşturur. Sert su kullanırken, elleriniz, saçlarınız veya çamaşırlarınız olsun, her şeyi temizlemek için daha fazla sabun veya deterjan gerekir. 

In [ ]:
fig = px.histogram(df, x="Hardness", y=Counter(df["Hardness"]), 
                   color="Potability", template="plotly_white", marginal="box",
                   barmode="group", histfunc="count", opacity=0.7, nbins=100, 
                   color_discrete_sequence = [colors_green[3], colors_blue[3]])

fig.add_vline(x=151, line_width=1, line_color=colors_dark[1], line_dash="dot", opacity=0.7)
fig.add_vline(x=301, line_width=1, line_color=colors_dark[1], line_dash="dot", opacity=0.7)
fig.add_vline(x=76, line_width=1, line_color=colors_dark[1], line_dash="dot", opacity=0.7)

fig.add_annotation(text="<76 mg/L<br> yumuşak kabul edilir", x=40, y=130, showarrow=False, font_size=9)
fig.add_annotation(text="76 ile 150 arası (mg/L)<br> orta derecede sert", x=113, y=130, showarrow=False, font_size=9)
fig.add_annotation(text="151 ile 300 (mg/L)<br> arası sert kabul edilir", x=250, y=130, showarrow=False, font_size=9)
fig.add_annotation(text=">300 mg/L<br> çok sert kabul edilir", x=340, y=130, showarrow=False, font_size=9)

fig.update_layout(font_family="monospace",
                  title=dict(text="Sertlik Dağılımı", x=0.53, y=0.95, font=dict(color=colors_dark[2], size=20)),
                  xaxis_title_text="Sertlik (mg/L)",
                  yaxis_title_text="Miktar",
                  legend=dict(x=1, y=0.96, bordercolor=colors_dark[4], borderwidth=0, tracegroupgap=5),
                  bargap=0.3)
fig.show()

* **pH Seviyesi:** Suyun pH'ı, asit-baz dengesinin bir ölçüsüdür ve çoğu doğal suda karbon dioksit-bikarbonat-karbonat denge sistemi tarafından kontrol edilir. Artan bir karbondioksit konsantrasyonu bu nedenle pH'ı düşürürken, bir azalma ise yükselmesine neden olur. Sıcaklık ayrıca dengeyi ve pH'ı da etkileyecektir. Saf suda, sıcaklık 25 °C artırıldığında pH'da yaklaşık 0,45'lik bir düşüş meydana gelir. Bikarbonat, karbonat ve hidroksil iyonları tarafından verilen tamponlama kapasitesine sahip suda, bu sıcaklık etkisi değiştirilir (APHA, 1989). Çoğu içme suyunun pH'ı 6.5-8.5 aralığındadır. Doğal sular, örneğin asit yağmuru veya kireçtaşı alanlarında daha yüksek pH'ın bir sonucu olarak daha düşük pH'a sahip olabilir.

In [ ]:
fig = px.histogram(df, x="ph", y=Counter(df["ph"]), color="Potability", template="plotly_white",
                   marginal="box", barmode="group", histfunc="count", opacity=0.7, nbins=100, 
                   color_discrete_sequence=[colors_green[3],colors_blue[3]])

fig.add_vline(x=7, line_width=1, line_color=colors_dark[1], line_dash="dot", opacity=0.7)

fig.add_annotation(text="<7 is Asidik", x=4, y=70, showarrow=False, font_size=10)
fig.add_annotation(text=">7 is Bazik", x=10, y=70, showarrow=False, font_size=10)

fig.update_layout(font_family="monospace",
                  title=dict(text="pH Seviyesi Dağılımı", x=0.5, y=0.95, font=dict(color=colors_dark[2], size=20)),
                  xaxis_title_text="pH Seviyesi",
                  yaxis_title_text="Miktar",
                  legend=dict(x=1, y=0.96, bordercolor=colors_dark[4], borderwidth=0, tracegroupgap=5),
                  bargap=0.3)
fig.show()

* **TDS:** TDS, suda çözünmüş parçacıkların veya katıların konsantrasyonu anlamına gelir. TDS, kalsiyum, magnezyum, klorürler, sülfatlar, bikarbonatlar vb. gibi inorganik tuzların yanı sıra suda kolayca çözünen daha birçok inorganik bileşikten oluşur. 

In [ ]:
fig = px.histogram(df, x="Solids", y=Counter(df["Solids"]), color="Potability", template="plotly_white",
                   marginal="box", barmode="group",histfunc="count", opacity=0.7, nbins=100, 
                   color_discrete_sequence=[colors_green[3],colors_blue[3]])

fig.update_layout(font_family="monospace",
                  title=dict(text="Toplam Çözünmüş Katıların Dağılımı", x=0.5, y=0.95, 
                             font=dict(color=colors_dark[2], size=20)),
                  xaxis_title_text="Çözünmüş Katılar (ppm)",
                  yaxis_title_text="Miktar",
                  legend=dict(x=1, y=0.96, bordercolor=colors_dark[4], borderwidth=0, tracegroupgap=5),
                  bargap=0.3)
fig.show()

* **Kloraminler:** Kloraminler (ikincil dezenfeksiyon olarak da bilinir), içme suyunu arıtmak için kullanılan dezenfektanlardır ve bunlar:

    * En yaygın olarak, içme suyunu arıtmak için klora amonyak eklendiğinde oluşur.
    * Su borulardan tüketicilere geçerken daha uzun süreli dezenfeksiyon sağlar.
    
Kloraminler 1930'lardan beri su hizmetleri tarafından kullanılmaktadır.

In [ ]:
fig = px.histogram(df, x="Chloramines", y=Counter(df["Chloramines"]), color="Potability", template="plotly_white",
                   marginal="box", barmode="group",histfunc="count", opacity=0.7, nbins=100, 
                   color_discrete_sequence=[colors_green[3],colors_blue[3]])

fig.add_vline(x=4, line_width=1, line_color=colors_dark[1], line_dash="dot", opacity=0.7)

fig.add_annotation(text="<4 ppm içmek<br> güvenli kabul edilir", x=1.8, y=90, showarrow=False)

fig.update_layout(font_family="monospace",
                  title=dict(text="Kloramin Dağılımı", x=0.53, y=0.95, font=dict(color=colors_dark[2], size=20)),
                  xaxis_title_text="Kloraminler (ppm)",
                  yaxis_title_text="Miktar",
                  legend=dict(x=1, y=0.96, bordercolor=colors_dark[4], borderwidth=0, tracegroupgap=5),
                  bargap=0.3)
fig.show()

* **Sülfat:** Sülfat (SO4) hemen hemen tüm doğal sularda bulunur. Çoğu sülfat bileşiğinin kaynağı, sülfit cevherlerinin oksidasyonu, şeyllerin varlığı veya endüstriyel atıklardır. Sülfat, yağmurun ana çözünmüş bileşenlerinden biridir. İçtiğimiz sudaki yüksek konsantrasyonlarda sülfat, sertliğin en yaygın iki bileşeni olan kalsiyum ve magnezyum ile birleştiğinde müshil etkisi olabilir. 

In [ ]:
fig = px.histogram(df, x="Sulfate", y=Counter(df["Sulfate"]), color="Potability", template="plotly_white",
                   marginal="box", barmode="group", histfunc="count", opacity=0.7, nbins=100,
                   color_discrete_sequence=[colors_green[3],colors_blue[3]])

fig.add_vline(x=250, line_width=1, line_color=colors_dark[1], line_dash="dot", opacity=0.7)

fig.add_annotation(text="<250 mg/L içmek<br> güvenli kabul edilir", x=175, y=90, showarrow=False)

fig.update_layout(font_family="monospace",
                  title=dict(text="Sülfat Dağılımı", x=0.53, y=0.95, font=dict(color=colors_dark[2], size=20)),
                  xaxis_title_text="Sülfat (mg/L)",
                  yaxis_title_text="Count",
                  legend=dict(x=1, y=0.96, bordercolor=colors_dark[4], borderwidth=0, tracegroupgap=5),
                  bargap=0.3)
fig.show()

* **İletkenlik:** İletkenlik, suyun bir elektrik akımını geçirme yeteneğinin bir ölçüsüdür. Çözünmüş tuzlar ve diğer inorganik kimyasallar elektrik akımını ilettiğinden, tuzluluk arttıkça iletkenlik artar. Yağ gibi organik bileşikler elektrik akımını çok iyi iletmezler ve bu nedenle suda düşük iletkenliğe sahiptirler. İletkenlik sıcaklıktan da etkilenir: su ne kadar sıcaksa iletkenlik o kadar yüksek olur.

In [ ]:
fig = px.histogram(df, x="Conductivity", y=Counter(df["Conductivity"]), color="Potability", template="plotly_white",
                   marginal="box", barmode="group", histfunc="count",
                   opacity=0.7, nbins=100, color_discrete_sequence=[colors_green[3],colors_blue[3]])

fig.add_annotation(text="İletkenlik aralığı hem (200-800)<br> hem de içilebilir ve içilebilir olmayan<br> sular için güvenlidir.",
                   x=600, y=90, showarrow=False)

fig.update_layout(font_family="monospace",
                  title=dict(text="Conductivity Distribution", x=0.5, y=0.95, font=dict(color=colors_dark[2], size=20)),
                  xaxis_title_text="Conductivity (μS/cm)",
                  yaxis_title_text="Count",
                  legend=dict(x=1, y=0.96, bordercolor=colors_dark[4], borderwidth=0, tracegroupgap=5),
                  bargap=0.3)
fig.show()

* **Organik Karbon:** Organik kirleticiler (doğal organik maddeler, böcek öldürücüler, herbisitler ve diğer tarımsal kimyasallar) yağış akışında su yollarına girer. Evsel ve endüstriyel atık sular da çeşitli miktarlarda organik kirleticilere katkıda bulunur. Kazara dökülmeler veya sızıntılar sonucunda endüstriyel organik atıklar nehirlere girebilir. Kirleticilerin bazıları arıtma işlemleriyle tamamen ortadan kaldırılamayabilir; bu nedenle içme suyu kaynakları için bir sorun haline gelebilirler. Bir su yolundaki organik içeriği bilmek önemlidir.

In [ ]:
fig = px.histogram(df, x="Organic_carbon", y=Counter(df["Organic_carbon"]), color="Potability", template="plotly_white",
                   marginal="box", barmode="group", histfunc="count", opacity=0.7, nbins=100,
                   color_discrete_sequence=[colors_green[3],colors_blue[3]])

fig.add_vline(x=10, line_width=1, line_color=colors_dark[1], line_dash="dot", opacity=0.7)

fig.add_annotation(text="Tipik Organik Karbon seviyesi<br> 10 ppm'ye kadar", x=5.3, y=110, showarrow=False)

fig.update_layout(font_family="monospace",
                  title=dict(text="Organik Karbon Dağılımı", x=0.5, y=0.95, font=dict(color=colors_dark[2], size=20)),
                  xaxis_title_text="Organik Karbon (ppm)",
                  yaxis_title_text="Miktar",
                  legend=dict(x=1, y=0.96, bordercolor=colors_dark[4], borderwidth=0, tracegroupgap=5),
                  bargap=0.3)
fig.show()

* **Trihalometanlar:** Trihalometanlar (THM'ler), musluk suyunu dezenfekte etmek için kullanılan klor ile sudaki doğal organik madde arasındaki reaksiyonun sonucudur. Yüksek seviyelerde, THM'ler kanser ve olumsuz üreme sonuçları gibi olumsuz sağlık etkileri ile ilişkilendirilmiştir.

In [ ]:
fig = px.histogram(df, x="Trihalomethanes", y=Counter(df["Trihalomethanes"]), color="Potability", 
                   template="plotly_white", marginal="box", barmode="group", histfunc="count",
                   opacity=0.7, nbins=100, color_discrete_sequence=[colors_green[3],colors_blue[3]])

fig.add_vline(x=80, line_width=1, line_color=colors_dark[1], line_dash="dot", opacity=0.7)

fig.add_annotation(text="Trihalometan seviyesinin üst sınırı 80 μg/L'dir.", x=115, y=90, showarrow=False)

fig.update_layout(font_family="monospace",
                  title=dict(text="Trihalometan Dağılımı", x=0.5, y=0.95, font=dict(color=colors_dark[2], size=20)),
                  xaxis_title_text="Trihalometanlar (μg/L)",
                  yaxis_title_text="Miktar",
                  legend=dict(x=1, y=0.96, bordercolor=colors_dark[4], borderwidth=0, tracegroupgap=5),
                  bargap=0.3)
fig.show()

* **Bulanıklık:** Bulanıklık, bir sıvının göreli berraklığının ölçüsüdür. Suyun optik bir özelliğidir ve su numunesinden bir ışık parlatıldığında sudaki malzeme tarafından saçılan ışık miktarının bir ölçüsüdür. Saçılan ışığın yoğunluğu ne kadar yüksek olursa, bulanıklık da o kadar yüksek olur. Suyun bulanık olmasına neden olan malzemeler arasında kil, silt, çok küçük inorganik ve organik maddeler, algler, çözünmüş renkli organik bileşikler ve plankton ve diğer mikroskobik organizmalar bulunur.

In [ ]:
fig = px.histogram(df, x="Turbidity", y=Counter(df["Turbidity"]), color="Potability", template="plotly_white",
                   marginal="box", barmode="group", histfunc="count", opacity=0.7, nbins=100, 
                   color_discrete_sequence=[colors_green[3],colors_blue[3]])

fig.add_vline(x=5, line_width=1, line_color=colors_dark[1], line_dash="dot", opacity=0.7)

fig.add_annotation(text="<5 NTU bulanıklık güvenli kabul edilir", x=6, y=90, showarrow=False)

fig.update_layout(font_family="monospace",
                  title=dict(text="Bulanıklık Dağılımı", x=0.5, y=0.95, font=dict(color=colors_dark[2], size=20)),
                  xaxis_title_text="Bulanıklık (NTU)",
                  yaxis_title_text="Miktar",
                  legend=dict(x=1, y=0.96, bordercolor=colors_dark[4], borderwidth=0, tracegroupgap=5),
                  bargap=0.3)
fig.show()

Scatter Plot Matrisi, tüm özellikler arasındaki korelasyonu bulmaya yardımcı olur.

In [ ]:
fig = px.scatter_matrix(df, df.drop("Potability", axis=1), height=1250, width=1250, template="plotly_white", 
                        opacity=0.7, color_discrete_sequence=[colors_blue[3],colors_green[3]], color="Potability",
                        symbol="Potability", color_continuous_scale=[colors_green[3],colors_blue[3]])

fig.update_layout(font_family="monospace", font_size=10, coloraxis_showscale=False,
                  legend=dict(x=0.02, y=1.07, bgcolor=colors_dark[4]),
                  title=dict(text="Scatter Plot Matrix b/w Features", x=0.5, y=0.97, 
                             font=dict(color=colors_dark[2], size=24)))
fig.show()

Gördüğümüz gibi, tüm özellikler arasında çok daha az korelasyon var gibi görünüyor.

In [ ]:
cor = df.drop("Potability", axis=1).corr()
cor

Korelasyonu görselleştirmek için bir Isı Haritası yapalım.

In [ ]:
fig = px.imshow(cor, height=800, width=800, color_continuous_scale=colors_blue, template="plotly_white")

fig.update_layout(font_family="monospace", 
                  title=dict(text="Correlation Heatmap", x=0.5, y=0.93, font=dict(color=colors_dark[2], size=24)),
                  coloraxis_colorbar=dict(len=0.85, x=1.1))
fig.show()

## Veri Ön İşleme

### 1. Eksik Değerler

In [ ]:
fig = msno.matrix(df, color=(0,0.5,0.5))

Veri setinde birçok eksik değer olduğunu görebiliyoruz.

In [ ]:
df.isnull().sum()

In [ ]:
df[df["Potability"]==0].describe()

In [ ]:
df[df["Potability"]==1].describe()

In [ ]:
df[df["Potability"]==0][["ph","Sulfate","Trihalomethanes"]].median()

In [ ]:
df[df["Potability"]==1][["ph","Sulfate","Trihalomethanes"]].median()

İçilebilir ve içilebilir olmayan suların ortalama ve medyan değerleri arasındaki farkın minimum olduğunu görebiliriz. Bu nedenle, değerleri uygulamak için özelliğin genel medyanını kullanırız.

In [ ]:
df["ph"].fillna(value=df["ph"].median(), inplace=True)
df["Sulfate"].fillna(value=df["Sulfate"].median(), inplace=True)
df["Trihalomethanes"].fillna(value=df["Trihalomethanes"].median(), inplace=True)
df.isnull().sum()

### 2. Standartlaştırma

In [ ]:
X = df.drop("Potability", axis=1).values
y = df["Potability"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Modelleme

### 1. Spot Checking

Nokta denetimi, verilerimiz için temel modelleri bulmak için harika bir yöntemdir. Oldukça kolay ve gerçekten daha az zaman alıyor.

In [ ]:
models =[
    ("SVC", SVC()),
    ("LR", LogisticRegression(max_iter=1000)), 
    ("KNN", KNeighborsClassifier(n_neighbors=10)),
    ("DTC", DecisionTreeClassifier()),
    ("GNB", GaussianNB()),
    ("SGDC", SGDClassifier()),
    ("Perc", Perceptron()),
    ("NC", NearestCentroid()),
    ("Ridge", RidgeClassifier()),
    ("NuSVC", NuSVC()),
    ("BNB", BernoulliNB()),
    ("RF", RandomForestClassifier()),
    ("ADA", AdaBoostClassifier()),
    ("XGB", GradientBoostingClassifier()),
    ("PAC", PassiveAggressiveClassifier()),
]

results = []
names = []
finalResults = []

for name,model in models:
    model.fit(X_train, y_train)
    model_results = model.predict(X_test)
    score = precision_score(y_test, model_results, average="macro")
    results.append(score)
    names.append(name)
    finalResults.append((name,score))
    
finalResults.sort(key=lambda k:k[1], reverse=True)
finalResults

En iyi 5 temel modeli seçtik ve hiperparametre ayarlaması yaptık.

In [ ]:
model_params = {
    "RF":{
        "model":RandomForestClassifier(),
        "params":{
            "n_estimators":[10,50,100,200],
            "max_features":["auto","sqrt","log2"],
            "max_depth":list(range(1,11)),
        }
    }
}

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2)

scores = []

for model_name,params in model_params.items():
    rs = RandomizedSearchCV(params["model"], params["params"], cv=cv, n_iter=20)
    rs.fit(X,y)
    scores.append([model_name, dict(rs.best_params_), rs.best_score_])

data = pd.DataFrame(scores, columns=["Model","Parameters","Score"])
data

Son birleştirilmiş model için en iyi parametrelere sahibiz.

## 2. Final Model

Modelleri daha iyi sonuçlar için birleştirmek için VotingClassfier'ı kullanıyoruz.

In [ ]:
param = data["Parameters"]

model = VotingClassifier(estimators = [
    ("RF", RandomForestClassifier(**param[0])),
], voting="hard")

accuracy=[]
scaler = StandardScaler()
skf = RepeatedStratifiedKFold(n_splits=5, n_repeats=2)
skf.get_n_splits(X,y)

for train_index, test_index in skf.split(X,y):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    model.fit(X_train,y_train)
    predictions = model.predict(X_test)
    score = accuracy_score(y_test,predictions)
    accuracy.append(score)
    
np.mean(accuracy)

In [ ]:
tuned_parameters = {
    "n_estimators": [100,200,500,1000], 
    "max_features": [3, 5, 7, 8],
    "min_samples_split": [2, 5, 10, 20],
}
model = RandomForestClassifier()
model_cv = GridSearchCV(model, tuned_parameters, cv=10, n_jobs=-1, verbose=2).fit(X_train, y_train)

In [ ]:
model_cv.best_params_

In [ ]:
model_tuned = RandomForestClassifier(max_features=5, min_samples_split=10, n_estimators=200).fit(X_train, y_train)
y_pred = model_tuned.predict(X_test)
accuracy_score(y_test, y_pred)